In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from vtk.numpy_interface import dataset_adapter as dsa
import numpy as np
from scipy.spatial.distance import cdist
from scipy.ndimage import binary_dilation, binary_closing
from scipy.ndimage import generate_binary_structure
from vmtk import vmtkscripts
from vmtk import vtkvmtk
import os
import sys
import numpy as np
import random
import networkx as nx
import sklearn
import vtk

In [2]:
surfr = vmtkscripts.vmtkSurfaceReader()
surfr.InputFileName = os.path.join(os.getcwd(), 'test-surf-no-hole.stl')
surfr.Execute()

surf = surfr.Surface

Reading STL surface file.


In [3]:
mc = vmtkscripts.vmtkMedialCurveCenterline()
mc.Surface = surf
# mc.Sigma = 1.5
# mc.Threshold = -15.0
# mc.PolyDataToImageDataSpacing = [0.2, 0.2, 0.2]
mc.Execute()

Converting Surface to Image Mask
Extracting Centerline Skeleton from Image Mask


In [4]:
converter = vmtkscripts.vmtkImageToNumpy()
converter.Image = mc.SkeletonImage
converter.Execute()

ad = converter.ArrayDict

wrapping vtkDataObject
setting origin
setting dimensions
setting spacing
writing point data: 


In [5]:
points = np.where(ad['PointData']['ImageScalars'] == 1)

In [6]:
origin = ad['Origin']
spacing = ad['Spacing']

stackedPoints = np.stack(points).T

ptCoords = origin + (stackedPoints * spacing)

In [7]:
vtkpoints = vtk.vtkPoints()

vtkcl = vtk.vtkPolyData()

In [8]:
for i in ptCoords:
    vtkpoints.InsertNextPoint(i)

In [9]:
vtkcl.SetPoints(vtkpoints)

In [10]:
pointIds = {}
locator = vtk.vtkPointLocator()
locator.SetDataSet(vtkcl)
locator.SetAutomatic(1)
for idx, coord in enumerate(ptCoords):
    closestPt = vtk.vtkIdList()
    locator.FindClosestNPoints(3, [coord[0], coord[1], coord[2]], closestPt)
    pointIds[idx] = [closestPt.GetId(1), closestPt.GetId(0), closestPt.GetId(2)]

In [11]:
cellDataArray = vtk.vtkCellArray()
for node, val in pointIds.items():
    numberOfCellPoints = len(val)
    cellDataArray.InsertNextCell(numberOfCellPoints)
    for cellPoint in val:
        cellDataArray.InsertCellPoint(cellPoint)
        
vtkcl.SetLines(cellDataArray)

In [25]:
import networkx as nx

In [28]:
G[59]

AtlasView({58: {}, 60: {}})

In [29]:
G[58]

AtlasView({57: {}, 59: {}, 60: {}})

In [30]:
G[60]

AtlasView({59: {}, 58: {}})

In [26]:
G = nx.from_dict_of_lists(pointIds)

In [27]:
pos = nx.nx_agraph.graphviz_layout(G)
nx.draw(G, pos=pos, with_labels=True)

/Users/rick/miniconda3/envs/vmtkplayground/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [12]:
cleaner = vtk.vtkCleanPolyData()
cleaner.SetInputData(vtkcl)
cleaner.ConvertLinesToPointsOn()
cleaner.ConvertPolysToLinesOn()
cleaner.ConvertStripsToPolysOn()
cleaner.Update()

In [13]:
viewer = vmtkscripts.vmtkCenterlineViewer()
viewer.Centerlines = cleaner.GetOutput()
# viewer.PointDataArrayName = 'RegionId'
viewer.Execute()

Quit renderer


In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

# Plot a sin curve using the x and y axes.
ax.scatter(points[0], points[1], points[2], zdir='z', label='curve in (x,y)')
ax.view_init(elev=20., azim=-35)

plt.show()

In [4]:
viewer = vmtkscripts.vmtkImageViewer()
viewer.Image = mc.SkeletonImage
viewer.Execute()

Quit renderer


In [ ]:
write = vmtkscripts.vmtkImageWriter()
write.Image = mc.SkeletonImage
write.OutputFileName = os.path.join(os.getcwd(), 'realout5.vti')
write.Execute()

### Previous Method

Link Edgels

In [ ]:
caster = vmtkscripts.vmtkImageCast()
caster.Image = mc.SkeletonImage
caster.OutputType = 'float'
caster.Execute()

# gradient the image
imgGradient = vtk.vtkImageGradient()
imgGradient.SetInputData(caster.Image)
imgGradient.SetDimensionality(3)
imgGradient.Update()

imgMagnitude = vtk.vtkImageMagnitude()
imgMagnitude.SetInputData(imgGradient.GetOutput())
imgMagnitude.Update()

# non maximum suppression
nonMax = vtk.vtkImageNonMaximumSuppression()
nonMax.SetMagnitudeInputData(imgMagnitude.GetOutput())
nonMax.SetVectorInputData(imgGradient.GetOutput())
nonMax.SetDimensionality(3)
nonMax.Update()

pad = vtk.vtkImageConstantPad()
pad.SetInputData(imgGradient.GetOutput())
pad.SetOutputNumberOfScalarComponents(3)
pad.SetConstant(0)
pad.Update()

i2sp1 = vtk.vtkImageToStructuredPoints()
i2sp1.SetInputData(nonMax.GetOutput())
i2sp1.SetVectorInputData(pad.GetOutput())
i2sp1.Update()
# link edgles
imgLink = vtk.vtkLinkEdgels()
imgLink.SetInputData(i2sp1.GetOutput())
imgLink.SetGradientThreshold(-1.5)
imgLink.SetPhiThreshold(180.0)
imgLink.Update()

o = imgLink.GetOutput()

In [ ]:
converter = vmtkscripts.vmtkIma

In [ ]:
lines = o.GetLines()

In [ ]:
from vtk.numpy_interface import dataset_adapter as dsa

In [ ]:
npo = dsa.WrapDataObject(o)

In [ ]:
npo.Points.shape

In [ ]:
centers = vtk.vtkCellCenters()
centers.SetInputData(o)
centers.VertexCellsOn()
centers.Update()

In [ ]:
clcent = centers.GetOutput()

In [ ]:
print(clcent)

In [ ]:
clcent.Ge

In [ ]:
viewer = vmtkscripts.vmtkCenterlineViewer()
viewer.Centerlines = regionnet
viewer.PointDataArrayName = 'RegionId'
viewer.Execute()

In [ ]:
imgLink.GetLinkThreshold()

In [ ]:
imgLink.GetGradientThreshold()

In [ ]:
imgLink.GetPhiThreshold()